# Master test notebook to run all user notebooks
    

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: January 30 2019

In [1]:
notebook_folder = 'H-Model-Development'

In [2]:
import os
import sys
import boto3

import aws.utils.notebooks.controller as controller
s3 = boto3.client('s3')

from aws.utils.notebooks.common import get_workspace,get_scratch_database
workspace = get_workspace()
notebook_bucket = workspace['NotebookOutputBucket']

In [3]:
# glue = boto3.client('glue')
# location = glue.get_database(Name='cms_raw_db')['Database']['LocationUri']
# bucket = location[5:].split('/')[0]
# my_session = boto3.session.Session()
# my_region = my_session.region_name
# my_region

In [5]:

!aws s3 ls $notebook_bucket/lake-user/output/notebooks/tests/$notebook_folder/ --recursive
!aws s3 rm s3://$notebook_bucket/lake-user/output/notebooks/tests/$notebook_folder/ --recursive 

In [6]:
!ls ..

A-LakeCreator	D-DataQuality	     H-Model-Development  ZZ-draft
B-DataAnalyst	E-DataMarts	     I-Model-Deployment
C-DataProfling	F-AdvanceStatistics  Z-Tests


In [7]:
%%bash --out output --err error -s "$notebook_folder"

find  "../$1" -type "f" -not -path '*/\.*' -name "*ipynb"

In [8]:
notebooks = output.split('\n')
notebooks

['../H-Model-Development/xgboost_mnist_2019-09-25/xgboost_mnist.ipynb', '']

In [9]:
notebooksToRun = []
for p in notebooks:
    if (len(p)<2):
        continue 
    parts = p.split('/') 
    folder = os.path.join(*parts[1:-1])
    notebookName = parts[-1]
    notebookToRun = {
        "tasks": [{
                  "notebookName": "{}".format(notebookName),
                  "sourcePath": "samples/notebooks/{}".format(folder),
                  "targetPath": "tests/{}".format(folder),
                  "params": {
                  }
                }]
    }
    notebooksToRun.append(notebookToRun)
notebooksToRun

[{'notebooks': [{'notebookName': 'xgboost_mnist.ipynb',
    'sourcePath': 'samples/notebooks/H-Model-Development/xgboost_mnist_2019-09-25',
    'targetPath': 'tests/H-Model-Development/xgboost_mnist_2019-09-25',
    'params': {}}]}]

In [22]:
containers = []
for n in notebooksToRun:
    container = controller.run_notebooks(n)
    containers = containers + container
containers

['arn:aws:ecs:us-east-1:495869084367:task/a99f4c98-b3f3-466b-8452-3fba84e766d2']

In [16]:
%%time 
controller.wait_for_tasks_to_complete(containers, 60,40)

In [17]:
controller.get_execution_history("samples/notebooks/Z-Tests", "run-model-dev-notebooks")

'No output notebooks founds at: s3://datalake-notebooksbucket-1beq2s8re8n3q/lake-user/output/notebooks/samples/notebooks/Z-Tests/run-model-dev-notebooks/'

### Lets check that all went well

In [18]:
%%bash --out output_notebooks -s "$notebook_folder"

aws s3 ls --recursive $notebook_bucket/lake-user/output/notebooks/tests/$1/ 

In [19]:
o_notebooks = []
for key in s3.list_objects_v2(Bucket=notebook_bucket, Prefix='lake-user/output/notebooks/tests/{}'.format(notebook_folder))['Contents']:
    path = key['Key']
    o_notebooks.append(path)
 
o_notebooks

['lake-user/output/notebooks/tests/H-Model-Development/xgboost_mnist_2019-09-25/xgboost_mnist/error@e1@20191001-14:40.ipynb',
 'lake-user/output/notebooks/tests/H-Model-Development/xgboost_mnist_2019-09-25/xgboost_mnist/error@e1@20191001-14:43.ipynb']

In [20]:
assert len(o_notebooks) == 4
for x in o_notebooks:
    if 'error@' in x:
        raise AssertionError('error in ' + x)

AssertionError: 

# End of notebook
